In [3]:
import pytz
import requests
import bs4
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import h5py
from io import BytesIO
import zipfile
import sys
import random
import pickle
import ENTSOE
import psycopg2 as pg
import sqlalchemy as sq
import logging
logging.disable()
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def get_connection():
    '''
    FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE AND RETURN THE SQLACHEMY ENGINE OBJECT
    -----------
    output: object
        SQLACHEMY ENGINE OBJECT - POSTGRESQL DATABASE CONNECTION
    '''
    user = 'postgres'
    password = 123
    host = 'localhost'
    port = 5432
    database = 'postgres'
    return sq.create_engine(url="postgresql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database))

DOMAIN_MAPPINGS = {
    'AL': '10YAL-KESH-----5',
    'AT': '10YAT-APG------L',
    'BA': '10YBA-JPCC-----D',
    'BE': '10YBE----------2',
    'BG': '10YCA-BULGARIA-R',
    'BY': '10Y1001A1001A51S',
    'CH': '10YCH-SWISSGRIDZ',
    'CZ': '10YCZ-CEPS-----N',
    'DE': '10Y1001A1001A83F',
    'DEp': '10Y1001A1001A63L',
    'DK': '10Y1001A1001A65H',
    'EE': '10Y1001A1001A39I',
    'ES': '10YES-REE------0',
    'FI': '10YFI-1--------U',
    'FR': '10YFR-RTE------C',
    'GB': '10YGB----------A',
    'GB-NIR': '10Y1001A1001A016',
    'GR': '10YGR-HTSO-----Y',
    'HR': '10YHR-HEP------M',
    'HU': '10YHU-MAVIR----U',
    'IE': '10YIE-1001A00010',
    'IT': '10YIT-GRTN-----B',
    'LT': '10YLT-1001A0008Q',
    'LU': '10YLU-CEGEDEL-NQ',
    'LV': '10YLV-1001A00074',
    # 'MD': 'MD',
    'ME': '10YCS-CG-TSO---S',
    'MK': '10YMK-MEPSO----8',
    'MT': '10Y1001A1001A93C',
    'NL': '10YNL----------L',
    'NO': '10YNO-0--------C',
    'PL': '10YPL-AREA-----S',
    'PT': '10YPT-REN------W',
    'RO': '10YRO-TEL------P',
    'RS': '10YCS-SERBIATSOV',
    'RU': '10Y1001A1001A49F',
    'RU-KGD': '10Y1001A1001A50U',
    'SE': '10YSE-1--------K',
    'SI': '10YSI-ELES-----O',
    'SK': '10YSK-SEPS-----K',
    'TR': '10YTR-TEIAS----W',
    'UA': '10YUA-WEPS-----0'
}

BIDDING_ZONES = DOMAIN_MAPPINGS.copy()
BIDDING_ZONES.update({
    'DE': '10Y1001A1001A63L',  # DE-AT-LU
    'LU': '10Y1001A1001A63L',  # DE-AT-LU
    'IT-NORD': '10Y1001A1001A73I',
    'IT-CNOR': '10Y1001A1001A70O',
    'IT-CSUD': '10Y1001A1001A71M',
    'IT-SUD': '10Y1001A1001A788',
    'IT-FOGN': '10Y1001A1001A72K',
    'IT-ROSN': '10Y1001A1001A77A',
    'IT-BRNN': '10Y1001A1001A699',
    'IT-PRGP': '10Y1001A1001A76C',
    'IT-SARD': '10Y1001A1001A74G',
    'IT-SICI': '10Y1001A1001A75E',
    'NO-1': '10YNO-1--------2',
    'NO-2': '10YNO-2--------T',
    'NO-3': '10YNO-3--------J',
    'NO-4': '10YNO-4--------9',
    'NO-5': '10Y1001A1001A48H',
    'SE-1': '10Y1001A1001A44P',
    'SE-2': '10Y1001A1001A45N',
    'SE-3': '10Y1001A1001A46L',
    'SE-4': '10Y1001A1001A47J',
    'DK-1': '10YDK-1--------W',
    'DK-2': '10YDK-2--------M'
})

TIMEZONE_MAPPINGS = {
    'AL': 'Europe/Tirane',
    'AT': 'Europe/Vienna',
    'BA': 'Europe/Sarajevo',
    'BE': 'Europe/Brussels',
    'BG': 'Europe/Sofia',
    'BY': 'Europe/Minsk',
    'CH': 'Europe/Zurich',
    'CZ': 'Europe/Prague',
    'DE': 'Europe/Berlin',
    'DEp': 'Europe/Berlin',
    'DK': 'Europe/Copenhagen',
    'EE': 'Europe/Talinn',
    'ES': 'Europe/Madrid',
    'FI': 'Europe/Helsinki',
    'FR': 'Europe/Paris',
    'GB': 'Europe/London',
    'GB-NIR': 'Europe/Belfast',
    'GR': 'Europe/Athens',
    'HR': 'Europe/Zagreb',
    'HU': 'Europe/Budapest',
    'IE': 'Europe/Dublin',
    'IT': 'Europe/Rome',
    'LT': 'Europe/Vilnius',
    'LU': 'Europe/Luxembourg',
    'LV': 'Europe/Riga',
    # 'MD': 'MD',
    'ME': 'Europe/Podgorica',
    'MK': 'Europe/Skopje',
    'MT': 'Europe/Malta',
    'NL': 'Europe/Amsterdam',
    'NO': 'Europe/Oslo',
    'PL': 'Europe/Warsaw',
    'PT': 'Europe/Lisbon',
    'RO': 'Europe/Bucharest',
    'RS': 'Europe/Belgrade',
    'RU': 'Europe/Moscow',
    'RU-KGD': 'Europe/Kaliningrad',
    'SE': 'Europe/Stockholm',
    'SI': 'Europe/Ljubljana',
    'SK': 'Europe/Bratislava',
    'TR': 'Europe/Istanbul',
    'UA': 'Europe/Kiev'
}

PSRTYPE_MAPPINGS = {
    'A03': 'Mixed',
    'A04': 'Generation',
    'A05': 'Load',
    'B01': 'Biomass',
    'B02': 'Fossil Brown coal/Lignite',
    'B03': 'Fossil Coal-derived gas',
    'B04': 'Fossil Gas',
    'B05': 'Fossil Hard coal',
    'B06': 'Fossil Oil',
    'B07': 'Fossil Oil shale',
    'B08': 'Fossil Peat',
    'B09': 'Geothermal',
    'B10': 'Hydro Pumped Storage',
    'B11': 'Hydro Run-of-river and poundage',
    'B12': 'Hydro Water Reservoir',
    'B13': 'Marine',
    'B14': 'Nuclear',
    'B15': 'Other renewable',
    'B16': 'Solar',
    'B17': 'Waste',
    'B18': 'Wind Offshore',
    'B19': 'Wind Onshore',
    'B20': 'Other',
    'B21': 'AC Link',
    'B22': 'DC Link',
    'B23': 'Substation',
    'B24': 'Transformer'}

DOCSTATUS = {'A05': 'Active', 'A09': 'Cancelled', 'A13': 'Withdrawn'}

BSNTYPE = {'A29': 'Already allocated capacity (AAC)',
           'A43': 'Requested capacity (without price)',
           'A46': 'System Operator redispatching',
           'A53': 'Planned maintenance',
           'A54': 'Unplanned outage',
           'A85': 'Internal redispatch',
           'A95': 'Frequency containment reserve',
           'A96': 'Automatic frequency restoration reserve',
           'A97': 'Manual frequency restoration reserve',
           'A98': 'Replacement reserve',
           'B01': 'Interconnector network evolution',
           'B02': 'Interconnector network dismantling',
           'B03': 'Counter trade',
           'B04': 'Congestion costs',
           'B05': 'Capacity allocated (including price)',
           'B07': 'Auction revenue',
           'B08': 'Total nominated capacity',
           'B09': 'Net position',
           'B10': 'Congestion income',
           'B11': 'Production unit'}

DOCUMENTTYPE = {'A09': 'Finalised schedule',
                'A11': 'Aggregated energy data report',
                'A25': 'Allocation result document',
                'A26': 'Capacity document',
                'A31': 'Agreed capacity',
                'A44': 'Price Document',
                'A61': 'Estimated Net Transfer Capacity',
                'A63': 'Redispatch notice',
                'A65': 'System total load',
                'A68': 'Installed generation per type',
                'A69': 'Wind and solar forecast',
                'A70': 'Load forecast margin',
                'A71': 'Generation forecast',
                'A72': 'Reservoir filling information',
                'A73': 'Actual generation',
                'A74': 'Wind and solar generation',
                'A75': 'Actual generation per type',
                'A76': 'Load unavailability',
                'A77': 'Production unavailability',
                'A78': 'Transmission unavailability',
                'A79': 'Offshore grid infrastructure unavailability',
                'A80': 'Generation unavailability',
                'A81': 'Contracted reserves',
                'A82': 'Accepted offers',
                'A83': 'Activated balancing quantities',
                'A84': 'Activated balancing prices',
                'A85': 'Imbalance prices',
                'A86': 'Imbalance volume',
                'A87': 'Financial situation',
                'A88': 'Cross border balancing',
                'A89': 'Contracted reserve prices',
                'A90': 'Interconnection network expansion',
                'A91': 'Counter trade notice',
                'A92': 'Congestion costs',
                'A93': 'DC link capacity',
                'A94': 'Non EU allocations',
                'A95': 'Configuration document',
                'B11': 'Flow-based allocations'}

def _extract_timeseries(xml_text):
    """
    Parameters
    ----------
    xml_text : str
    Yields
    -------
    bs4.element.tag
    """
    if not xml_text:
        return
    soup = bs4.BeautifulSoup(xml_text, 'html.parser')
    for timeseries in soup.find_all('timeseries'):
        yield timeseries

def _parse_generation_forecast_timeseries(soup):
    """
    Parameters
    ----------
    soup : bs4.element.tag
    Returns
    -------
    pd.Series
    """
    psrtype = soup.find('psrtype').text
    positions = []
    quantities = []
    for point in soup.find_all('point'):
        positions.append(int(point.find('position').text))
        quantities.append(float(point.find('quantity').text))

    series = pd.Series(index=positions, data=quantities)
    series = series.sort_index()
    series.index = _parse_datetimeindex(soup)

    series.name = PSRTYPE_MAPPINGS[psrtype]
    return series

def _parse_datetimeindex(soup):
    """
    Create a datetimeindex from a parsed beautifulsoup,
    given that it contains the elements 'start', 'end'
    and 'resolution'
    Parameters
    ----------
    soup : bs4.element.tag
    Returns
    -------
    pd.DatetimeIndex
    """
    start = pd.Timestamp(soup.find('start').text)
    end = pd.Timestamp(soup.find('end').text)
    delta = _resolution_to_timedelta(res_text=soup.find('resolution').text)
    index = pd.date_range(start=start, end=end, freq=delta, closed='left')
    return index

def _resolution_to_timedelta(res_text: str) -> str:
    """
    Convert an Entsoe resolution to something that pandas can understand
    """
    resolutions = {
        'PT60M': '60min',
        'P1Y': '12M',
        'PT15M': '15min',
        'PT30M': '30min'
    }
    delta = resolutions.get(res_text)
    if delta is None:
        raise NotImplementedError("Sorry, I don't know what to do with the "
                                  "resolution '{}', because there was no "
                                  "documentation to be found of this format. "
                                  "Everything is hard coded. Please open an "
                                  "issue.".format(res_text))
    return delta

def data_arrange(generation):
    data_table = generation

    set_columns = generation.columns
    set_gen = ['Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil',
               'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage',
               'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore', 'Other', 'Biomass']

    for kind in set_gen:
        if not kind in set_columns:
            data_table[kind] = [0]*(data_table.shape[0])
    
    EFt = (71*data_table['Biomass']+800*(data_table['Fossil Hard coal'] + data_table['Fossil Coal-derived gas']) +400*data_table['Fossil Gas']+520*data_table['Fossil Oil']+520*data_table['Fossil Oil shale'] + 376*data_table['Fossil Peat'] + 45*data_table['Geothermal']+ 34*data_table['Hydro Pumped Storage'] + 4*data_table['Hydro Run-of-river and poundage']+
    9.0*data_table['Hydro Water Reservoir']+  820*data_table['Fossil Brown coal/Lignite'] + 376*data_table['Marine'] + 11*data_table['Nuclear'] + 43*data_table['Solar'] + 690*data_table['Waste'] + 
    9*data_table['Wind Offshore'] + 8*data_table['Wind Onshore']+ 33*data_table['Other renewable'] + 376*data_table['Other'])/(data_table.sum(axis=1))

    return EFt

def _datetime_to_str(dtm):
    """
    Convert a datetime object to a string in UTC
    of the form YYYYMMDDhh00
    Parameters
    ----------
    dtm : pd.Timestamp
        Recommended to use a timezone-aware object!
        If timezone-naive, UTC is assumed
    Returns
    -------
    str
    """
    if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
        dtm = dtm.tz_convert("UTC")
    fmt = '%Y%m%d%H00'
    ret_str = dtm.strftime(fmt)
    return ret_str

def base_request(params,api_key,start,end):
    start_str = _datetime_to_str(start)
    end_str = _datetime_to_str(end)
    base_params = {
        'securityToken': api_key,
        'periodStart': start_str,
        'periodEnd': end_str
    }
    params.update(base_params)
    retry_count = 100
    session = requests.Session()
    error = None
    for _ in range(retry_count):
        response = session.get(url=URL,params=params)
        try:
            response.raise_for_status()
        except requests.HTTPError as e:
            error = e
            soup = BeautifulSoup(response.text,'html.parser')
            text = soup.find_all('text')
            if len(text):
                error_text = soup.find('text').text
                if 'No matching data found' in error_text:
                    return None
            retry_delay = random.randint(1,180)
            print("HTTP Error, retrying in {} seconds".format(retry_delay))
            sleep(retry_delay)
        else:
            return response
    else:
        raise error

In [6]:
from datetime import date, datetime, timedelta
from tqdm import tqdm
import numpy as np
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
days=[]
for result in tqdm(perdelta(date(2019,1, 1), date(2022, 1, 1), timedelta(days=1))):
    today=str(result)
    today=today.replace('-','/')
    t2=today
    if today[5] == '0':
        t2=str(t2[0:5])+str(t2[6:])
        if today[8] == '0':
            t2=str(t2[0:7])+str(t2[8:])
    if today[8] == '0':
        t2=str(t2[0:8])+str(t2[9:])
    days.append(t2)

1096it [00:00, 293941.89it/s]


In [ ]:
paises = ['Germany', 'Portugal', 'Spain', 'Belgium']
codigos = ['DE', 'PT','ES', 'BE']
key = '' #Restful API access: use your key aunthentication.
for p in range(len(paises)):
    pais = paises[p]
    cod = codigos[p]
    datatotal= pd.DataFrame()
    for d in tqdm(range(len(days)-1)):
        """until 25 march inc, put the desired date and +01:00; otherwise +02:00, until october 28 inc. SKIP 26/03 and 29/10"""
        #query = ENTSOE.Entsoe('747061e5-91e4-4fc5-a722-659f33b144b6') #my key
        datelist_start = pd.date_range(start=days[d]+str(' ')+str('00:00:00 +01:00'),end=days[d+1]+str(' ')+str('23:45:00 +01:00'),freq='D',name='datelist_start')
        #period of interest. it cannot be too long, otherwise it will give you an error
        filename = "original_datasets/Data_2019_2021"+str(pais)

        realized = pd.DataFrame()
        generation = pd.DataFrame()
        emission = pd.DataFrame()
        #session = requests.Session()
        URL = 'https://transparency.entsoe.eu/api'
        #print("\n")
        query = ENTSOE.Entsoe(key) #my key
        """until 25 march inc, put the desired date and +01:00; otherwise +02:00, until october 28 inc. SKIP 26/03 and 29/10"""
        consumption = []
        #for k in datelist_start:
        k=datelist_start[0]
        #query.query_generation('DE',k,k+pd.DateOffset(hours=24),True) #country; split data every 24hours; parse the data
        country_code = cod
        start = k
        end = k+pd.DateOffset(hours=24)
        '''params = {
            'documentType': 'A75',
            'processType': 'A16',
            'in_Domain': DOMAIN_MAPPINGS[country_code],
            'securityToken': key,
            'periodStart': start_str,
            'periodEnd': end_str
        }'''
        params = {
            'documentType': 'A75',
            'processType': 'A16',
            'in_Domain': DOMAIN_MAPPINGS[country_code],
        }
        #response = session.get(url=URL, params=params)
        var = False
        while var == False:
            try:
                response = base_request(params,key,start,end)
                var = True
            except:
                var = False

        #print("\n"+response.text+"\n")
        #if response is None:
        #    print("Resposta é None")
        all_series = {}
        for soup in _extract_timeseries(response.text):
            ts = _parse_generation_forecast_timeseries(soup)
            series = all_series.get(ts.name)
            if series is None:
                all_series[ts.name] = ts
            else:
                series = series.append(ts)
                series.sort_index()
                all_series[series.name] = series

        for name in all_series:
            ts = all_series[name]
            all_series[name] = ts[~ts.index.duplicated(keep='first')]

        gen_all = pd.DataFrame.from_dict(all_series)
        gen_all = gen_all.tz_convert(TIMEZONE_MAPPINGS[country_code])
        production = gen_all
        #print("\n"+str(production.shape[0])+"\n")
        EF = data_arrange(production)   #calculate emission factors
        gen_all = gen_all.resample('60T').mean()
        EF = EF.resample('60T').mean()
        generation = generation.append(gen_all)

        emission = emission.append(pd.DataFrame(EF,columns=['Emission']))
        query.query_consumption(cod, k, k + pd.DateOffset(hours = 24), True)
        load = query.load.resample('60T').mean()
        t=load.to_list()
        consumption.extend(t)

        #gen_sum = gen_all.sum(axis='columns') #total generation
        #print(".",end="")
        #sys.stdout.flush()
        generation.insert(20, "Emission",emission['Emission'], True) 
        generation['Consumption']=consumption
        datatotal=datatotal.append(generation)
    out_file = open(filename+".pickle",'wb')
    datatotal['Date']=datatotal.index.strftime("%Y/%m/%d")
    datatotal['Hour']=datatotal.index.strftime("%H:%M")
    datatotal['UTC']=datatotal.index.strftime("%z")
    datatotal = datatotal.reset_index(drop = True)
    pickle.dump(datatotal,out_file)
    out_file.close()
    datatotal.to_sql(name=str.lower(pais), con = get_connection(),schema = 'original_data', if_exists = 'replace', chunksize = None, index = False)
    print("finish")